In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd
import numpy as np
import cv2
from typing import Tuple, List
import os
import sys
from tqdm import tqdm

In [2]:
print(f'Tensorflow version: {tf.__version__}')
print(f'Pandas version: {pd.__version__}')
print(f'NumPy version: {np.__version__}')
print(f'OpenCV version: {cv2.__version__}')
!python --version

Tensorflow version: 2.3.1
Pandas version: 1.1.1
NumPy version: 1.18.5
OpenCV version: 4.4.0


Python 3.6.10 :: Anaconda, Inc.


In [3]:
"""
We turn this on to prevent tensorflow from throwing a fit about
things that take longer than the batch training in the model.fit
call (namely, the tensorboard callback can take more time to
execute than the batch training iteration itself).

NOTE: This can be disabled simply by commenting it out. If you
think there is a weird tensorflow issue happening, do that to see
the full tensorflow logs during runtime.
"""
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

## Utility Functions

In [4]:
def getDirsAndClasses(root: str, file: str) -> Tuple[List[str], List[int]]:
    imageDirs = []
    classes = []
    line = ""
    with open(root + file, "r") as f:
        for line in tqdm(f):
            imageDir, clazz = line.split()
            imageDirs.append(imageDir)
            classes.append(int(clazz))
    return imageDirs, classes

In [5]:
def preprocessImage(img: np.ndarray) -> tf.Tensor:
    return img.astype(np.float16) / 255.0

In [6]:
def makeOneHot(value: int, size: int) -> np.ndarray:
    onehot = np.zeros(size)
    onehot[value] = 1
    return onehot

In [7]:
def balanceData(data: pd.DataFrame) -> pd.DataFrame:
    data = data.groupby("class")
    data = data.apply(lambda x: x.sample(data.size().min()).reset_index(drop=True))
    return data.reset_index(drop=True)

In [28]:
def makeModel(inputShape: (224, 224, 3)) -> keras.Model:
    inputs = keras.Input(shape=inputShape, name="Input")
    
    x = layers.Conv2D(filters=16, kernel_size=11, strides=4, activation="relu")(inputs)
    x = layers.MaxPooling2D(pool_size=3, strides=2)(x)
    x = layers.Conv2D(filters=20, kernel_size=5, strides=1, activation="relu")(x)
    x = layers.MaxPooling2D(pool_size=3, strides=2)(x)
    x = layers.Conv2D(filters=30, kernel_size=3, strides=1, activation="relu")(x)
    x = layers.MaxPooling2D(pool_size=3, strides=2)(x)
    x = layers.Dense(units=48, activation="relu")(x)
    x = layers.Flatten()(x)
    outputs = layers.Dense(units=2, activation="softmax")(x)
    
    model = keras.Model(inputs=inputs, outputs=outputs, name="mAlexNet")
    return model

## Global Variables

In [9]:
callbacks = [
    tf.keras.callbacks.EarlyStopping(
        monitor="accuracy",
        min_delta=0.01,
        patience=3,
        verbose=1
    ),
    tf.keras.callbacks.TensorBoard(
        log_dir="logs/mAlexNet",
        write_graph=True,
        write_images=True
    )
]

root = os.getcwd() + "\\Data\\CNR-EXT-150x150"
# train_imageDirs, train_classes = getDirsAndClasses(root, "\\LABELS\\train.txt")
# test_imageDirs, test_classes = getDirsAndClasses(root, "\\LABELS\\test.txt")
# val_imageDirs, val_classes = getDirsAndClasses(root, "\\LABELS\\val.txt")
imageDirs, classes = getDirsAndClasses(root, "\\LABELS\\all.txt")

classDict = {
    0: "free",
    1: "busy"
}

batchSize = 32

144965it [00:00, 869905.32it/s]


## Data Acquisition and Preprocessing

In [10]:
data = pd.DataFrame([
            {
                "image": root + "\\PATCHES\\" + filename,
                #"image": preprocessImage(cv2.resize(cv2.imread(root + "\\PATCHES\\" + filename), dsize=(224, 224), interpolation=cv2.INTER_AREA)),
                "class": clazz,
                "weather": filename[0]
            }
            for filename, clazz in tqdm(zip(imageDirs, classes))
        ])

144965it [00:00, 1212062.03it/s]


In [11]:
data.groupby("class")["class"].value_counts()

class  class
0      0        65658
1      1        79307
Name: class, dtype: int64

In [12]:
data.groupby("weather")["weather"].value_counts()

weather  weather
O        O          44243
R        R          37544
S        S          63178
Name: weather, dtype: int64

In [13]:
data.groupby(["class", "weather"])["class"].value_counts()

class  weather  class
0      O        0        21067
       R        0        18926
       S        0        25665
1      O        1        23176
       R        1        18618
       S        1        37513
Name: class, dtype: int64

In [14]:
classes = list(data.groupby("class").groups.keys())

In [15]:
data["onehot"] = data["class"].apply(
    func=lambda x: makeOneHot(classes.index(x), len(classes))
)

### Split the data into train and test subsets

In [16]:
train = data.groupby("class").sample(frac=0.8)
train.groupby("class")["class"].value_counts()

class  class
0      0        52526
1      1        63446
Name: class, dtype: int64

In [17]:
test = data.drop(train.index).reset_index(drop=True)
test.groupby("class")["class"].value_counts()

class  class
0      0        13132
1      1        15861
Name: class, dtype: int64

In [18]:
train = train.reset_index(drop=True)

In [19]:
train = tf.data.Dataset.from_tensor_slices(
    (
        np.array([preprocessImage(cv2.resize(cv2.imread(image), dsize=(224, 224), interpolation=cv2.INTER_AREA)) for image in tqdm(train["image"].values.tolist())]),
        np.array(train["onehot"].values.tolist())
    )
).shuffle(
    buffer_size=len(train),
    reshuffle_each_iteration=True
).batch(batchSize)

100%|█████████████████████████████████| 115972/115972 [04:18<00:00, 449.28it/s]


In [20]:
test = tf.data.Dataset.from_tensor_slices(
    (
        np.array([preprocessImage(cv2.resize(cv2.imread(image), dsize=(224, 224), interpolation=cv2.INTER_AREA)) for image in tqdm(test["image"].values.tolist())]),
        np.array(test["onehot"].values.tolist())
    )
).shuffle(
    buffer_size=len(test),
    reshuffle_each_iteration=True
).batch(batchSize)

100%|███████████████████████████████████| 28993/28993 [01:15<00:00, 382.32it/s]


## Model Definition

In [29]:
model = makeModel(inputShape=(224, 224, 3))

In [30]:
model.compile(
    optimizer="adam",
    loss=keras.losses.CategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"]
)

In [31]:
model.summary()

Model: "mAlexNet"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Input (InputLayer)           [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 54, 54, 16)        5824      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 26, 26, 16)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 22, 22, 20)        8020      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 10, 10, 20)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 8, 8, 30)          5430      
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 3, 3, 30)          0  

## Training and Evaluation

In [32]:
model.fit(
    train,
    epochs=100,
    callbacks=callbacks
)

Epoch 1/100
 319/3625 [=>............................] - ETA: 3:22 - loss: 0.4300 - accuracy: 0.8742

KeyboardInterrupt: 

In [ ]:
loss, accuracy = model.evaluate(test)

In [ ]:
model.predict(test)

## Results

In [ ]:
print(f'Test Loss: {loss}\nTest Accuracy: {accuracy}')

In [ ]:
model.save("Models/mAlexNet")